# 🛠️ Lab 7: Tool Orchestration
## Module 5 - Multi-Tool Workflows for Banking Agents

**Duration:** 25 minutes

**Objectives:**
- Implement multi-tool agent workflows
- Apply tool permission levels
- Build guardrails for financial operations

In [ ]:
!pip install openai -q

In [ ]:
import os, json
from datetime import datetime
from enum import Enum
from functools import wraps

DEMO_MODE = False
client = None
MODEL_NAME = "gpt-4o"

try:
    from google.colab import userdata
    AZURE_OPENAI_KEY = userdata.get('AZURE_OPENAI_KEY')
    AZURE_OPENAI_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')
    try: MODEL_NAME = userdata.get('AZURE_OPENAI_DEPLOYMENT')
    except: pass
    if AZURE_OPENAI_KEY and AZURE_OPENAI_ENDPOINT:
        if not AZURE_OPENAI_ENDPOINT.startswith('http'):
            AZURE_OPENAI_ENDPOINT = 'https://' + AZURE_OPENAI_ENDPOINT
        print(f"✅ Loaded. Model: {MODEL_NAME}")
    else: raise ValueError()
except: print("⚠️ DEMO MODE"); DEMO_MODE = True

if not DEMO_MODE:
    from openai import AzureOpenAI
    client = AzureOpenAI(api_key=AZURE_OPENAI_KEY, api_version="2024-06-01", azure_endpoint=AZURE_OPENAI_ENDPOINT)

## Part 1: Tool Permission Levels

| Level | Type | Examples | Guardrails |
|-------|------|----------|------------|
| 1 | Read-Only | Balance, history | None |
| 2 | Write | Create ticket | Logging |
| 3 | Financial | Transfer | Approval |

In [ ]:
class ToolPermission(Enum):
    READ_ONLY = 1
    WRITE = 2
    FINANCIAL = 3

AUDIT_LOG = []

def require_permission(permission):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            if permission == ToolPermission.FINANCIAL and not kwargs.get("approved"):
                print(f"⚠️ {func.__name__} requires approval")
                return {"status": "pending_approval"}
            result = func(*args, **kwargs)
            if permission != ToolPermission.READ_ONLY:
                AUDIT_LOG.append({"tool": func.__name__, "time": datetime.now().isoformat()})
            return result
        return wrapper
    return decorator

print("✅ Permission system defined")

In [ ]:
@require_permission(ToolPermission.READ_ONLY)
def get_account_balance(customer_id): return {"checking": 5420.50, "savings": 12350.00}

@require_permission(ToolPermission.READ_ONLY)
def get_login_history(customer_id): return {"recent": [{"location": "San Francisco"}, {"location": "Lagos, Nigeria"}]}

@require_permission(ToolPermission.WRITE)
def create_fraud_case(customer_id, risk_level, description): return {"case_id": "FRAUD-001", "status": "open"}

@require_permission(ToolPermission.FINANCIAL)
def block_account(customer_id, reason, approved=False): return {"status": "blocked"}

TOOL_FUNCTIONS = {"get_account_balance": get_account_balance, "get_login_history": get_login_history, 
                  "create_fraud_case": create_fraud_case, "block_account": block_account}

print("✅ Tools defined")

## Part 2: Multi-Tool Agent

In [ ]:
TOOLS = [
    {"type": "function", "function": {"name": "get_account_balance", "description": "Get balance", "parameters": {"type": "object", "properties": {"customer_id": {"type": "string"}}, "required": ["customer_id"]}}},
    {"type": "function", "function": {"name": "get_login_history", "description": "Get logins", "parameters": {"type": "object", "properties": {"customer_id": {"type": "string"}}, "required": ["customer_id"]}}},
    {"type": "function", "function": {"name": "create_fraud_case", "description": "Create fraud case", "parameters": {"type": "object", "properties": {"customer_id": {"type": "string"}, "risk_level": {"type": "string"}, "description": {"type": "string"}}, "required": ["customer_id", "risk_level", "description"]}}}
]

In [ ]:
def run_demo_investigation(alert):
    print("\n⚠️ DEMO MODE\n")
    print("🔧 get_account_balance → checking: $5,420")
    print("🔧 get_login_history → San Francisco, Lagos Nigeria")
    print("🔧 create_fraud_case → FRAUD-001 created")
    return "RECOMMENDATION: BLOCK - Login from unusual location (Nigeria) with high-value transaction attempt."

def run_fraud_investigation(alert, max_iter=10):
    print("\n🔍 FRAUD INVESTIGATION")
    print(f"Alert: {alert[:50]}...\n")
    
    if DEMO_MODE or not client:
        return run_demo_investigation(alert)
    
    messages = [{"role": "system", "content": "Fraud investigator. Gather info, analyze, recommend."}, 
                {"role": "user", "content": f"Investigate: {alert}"}]
    
    try:
        for i in range(max_iter):
            response = client.chat.completions.create(model=MODEL_NAME, messages=messages, tools=TOOLS)
            msg = response.choices[0].message
            messages.append(msg)
            
            if not msg.tool_calls:
                return msg.content
            
            for tc in msg.tool_calls:
                name = tc.function.name
                args = json.loads(tc.function.arguments)
                print(f"🔧 {name}")
                result = TOOL_FUNCTIONS[name](**args) if name in TOOL_FUNCTIONS else {"error": "Unknown"}
                messages.append({"role": "tool", "tool_call_id": tc.id, "content": json.dumps(result)})
        return "Max iterations"
    except Exception as e:
        print(f"⚠️ Error: {e}")
        return run_demo_investigation(alert)

In [ ]:
alert = """FRAUD ALERT - Customer C-789
Login from Lagos, Nigeria + $45,000 wire attempt
Usual location: San Francisco"""

result = run_fraud_investigation(alert)
print("\n" + "="*50)
print("FINAL REPORT:")
print(result)

In [ ]:
print("\n📋 AUDIT LOG:")
for entry in AUDIT_LOG:
    print(f"  {entry['time']}: {entry['tool']}")

## ✅ Lab 7 Complete!

**Key Takeaways:**
- Tools must be categorized by permission level
- Financial operations require human approval
- All write operations should be logged

**Next:** `08_resilience_patterns.ipynb`